<a href="https://colab.research.google.com/github/jsansao/transformers_pt/blob/main/BERTimbau_Fine_tuning_CETENFolha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inferência usando transformers pré-treinados do HuggingFace 

BERTimbau
CENTENFolha - 8 partes
Batch size = 16
GPU T4x2

In [1]:
#@title Passo 1:Instalando Hugging Face Transformers
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install transformers datasets
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.9.1
# tokenizers version at notebook update --- 0.7.0

Found existing installation: tensorflow 2.6.4
Uninstalling tensorflow-2.6.4:
  Successfully uninstalled tensorflow-2.6.4
tokenizers                            0.12.1
transformers                          4.20.1


In [2]:
#@title Step 1: Loading the Dataset
#1.Load kant.txt using the Colab file manager
#2.Downloading the file from GitHub
#!curl -L  https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/dump_Machado_Nobreak.txt --output "dump.txt"
!curl -L https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_aa --output "dump.txt"
#!curl -L https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/Lyrics_ChicoBuarque.txt --output "kant.txt"

!awk NF < dump.txt > kant.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3724k  100 3724k    0     0  7727k      0 --:--:-- --:--:-- --:--:-- 7711k


In [3]:
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_aa 
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ab
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ac
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ad
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ae 
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_af
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ag
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ah

--2022-10-29 00:55:37--  https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_aa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3813899 (3.6M) [text/plain]
Saving to: ‘cetenfolha_aa’

cetenfolha_aa       100%[===================>]   3.64M  --.-KB/s    in 0.07s   

2022-10-29 00:55:38 (52.0 MB/s) - ‘cetenfolha_aa’ saved [3813899/3813899]

--2022-10-29 00:55:39--  https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3813899 (3.6M) [text/p

In [4]:
!cat cetenfolha_aa cetenfolha_ab cetenfolha_ac cetenfolha_ad cetenfolha_ae cetenfolha_af cetenfolha_ag cetenfolha_ah>  dump.txt
!awk NF < dump.txt > kant.txt

In [5]:
#@title Passo 2:Baixando e salvando BR_BERTo
#https://huggingface.co/rdenadai/BR_BERTo

from transformers import AutoTokenizer, AutoModelForMaskedLM

#tokenizer = AutoTokenizer.from_pretrained("rdenadai/BR_BERTo")
#model = AutoModelForMaskedLM.from_pretrained("rdenadai/BR_BERTo")

tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
model = AutoModelForMaskedLM.from_pretrained('neuralmind/bert-base-portuguese-cased')

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
from datasets import load_dataset
#datasets = load_dataset("text", data_files={"train": "kant.txt", "validation": "kant_teste.txt"})
#datasets = load_dataset("text", data_files={"train": "kant.txt"}, split='train[:90%]')
ds = load_dataset("text", data_files={"train": "kant.txt"})

datasets = ds["train"].train_test_split()

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-20f1a5c3a14eeedc/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [8]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#0:   0%|          | 0/60 [00:00<?, ?ba/s]

#1:   0%|          | 0/60 [00:00<?, ?ba/s]

#2:   0%|          | 0/60 [00:00<?, ?ba/s]

#3:   0%|          | 0/60 [00:00<?, ?ba/s]

#0:   0%|          | 0/20 [00:00<?, ?ba/s]

#1:   0%|          | 0/20 [00:00<?, ?ba/s]

#2:   0%|          | 0/20 [00:00<?, ?ba/s]

#3:   0%|          | 0/20 [00:00<?, ?ba/s]

exemplo de entrada

In [9]:
tokenized_datasets["train"][4]

{'input_ids': [101,
  1643,
  131,
  1000,
  1448,
  22281,
  118,
  21248,
  117,
  1000,
  275,
  22296,
  3708,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [10]:
#block_size = tokenizer.model_max_length
block_size = 32

In [11]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [12]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/60 [00:00<?, ?ba/s]

#1:   0%|          | 0/60 [00:00<?, ?ba/s]

#2:   0%|          | 0/60 [00:00<?, ?ba/s]

#3:   0%|          | 0/60 [00:00<?, ?ba/s]

#0:   0%|          | 0/20 [00:00<?, ?ba/s]

#1:   0%|          | 0/20 [00:00<?, ?ba/s]

#2:   0%|          | 0/20 [00:00<?, ?ba/s]

#3:   0%|          | 0/20 [00:00<?, ?ba/s]

In [13]:
#@title Step 11: Defining a Data Collator
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [14]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {metric.compute(predictions=predictions, references=labels)}

In [15]:
#@title Step 12: Initializing the Trainer
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./BERTimbau-CENTENFolha-8_parts",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],    
)

In [16]:
#@title Step 13: Pre-training the Model
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 173367
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 54180
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,2.425600
1000,2.361600
1500,2.350600
2000,2.292200
2500,2.281700
3000,2.280300
3500,2.262900
4000,2.261400
4500,2.234800
5000,2.240400


Saving model checkpoint to ./BERTimbau-CENTENFolha-8_parts/checkpoint-10000
Configuration saved in ./BERTimbau-CENTENFolha-8_parts/checkpoint-10000/config.json
Model weights saved in ./BERTimbau-CENTENFolha-8_parts/checkpoint-10000/pytorch_model.bin
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Saving model checkpoint to ./BERTimbau-CENTENFolha-8_parts/checkpoint-20000
Configuration saved in ./BERTimbau-CENTENFolha-8_parts/checkpoint-20000/config.json
Model weights saved in ./BERTimbau-CENTENFolha-8_parts/checkpoint-20000/pytorch_model.bin
/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was

TrainOutput(global_step=54180, training_loss=1.9523954738590188, metrics={'train_runtime': 13377.0031, 'train_samples_per_second': 129.601, 'train_steps_per_second': 4.05, 'total_flos': 2.851915066247808e+16, 'train_loss': 1.9523954738590188, 'epoch': 10.0})

In [17]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 57683
  Batch size = 16


Perplexity: 6.33


In [18]:
eval_results

{'eval_loss': 1.8451666831970215,
 'eval_runtime': 213.596,
 'eval_samples_per_second': 270.057,
 'eval_steps_per_second': 16.882,
 'epoch': 10.0}

In [19]:
#@title Step 14: Saving the Final Model(+tokenizer + config) to disk
trainer.save_model("./BERTimbau-CENTENFolha-8_parts")

Saving model checkpoint to ./BERTimbau-CENTENFolha-8_parts
Configuration saved in ./BERTimbau-CENTENFolha-8_parts/config.json
Model weights saved in ./BERTimbau-CENTENFolha-8_parts/pytorch_model.bin


In [20]:
# Saving model on Wandb
import wandb
wandb.save('BERTimbau-CENTENFolha-8_parts.h5')

[]

In [21]:
#@title Step 3: Configurando o pipeline fill-mask
#@title Step 15: Language Modeling with the FillMaskPipeline
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./BERTimbau-CENTENFolha-8_parts",
    tokenizer=tokenizer
)

loading configuration file ./BERTimbau-CENTENFolha-8_parts/config.json
Model config BertConfig {
  "_name_or_path": "./BERTimbau-CENTENFolha-8_parts",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 29794
}

loading configuration file ./BERTimbau-CENTENFo

In [22]:
fill_mask("O rapaz olhou para o [MASK] ")

[{'score': 0.16590942442417145,
  'token': 2979,
  'token_str': 'alto',
  'sequence': 'O rapaz olhou para o alto'},
 {'score': 0.15819883346557617,
  'token': 8242,
  'token_str': 'céu',
  'sequence': 'O rapaz olhou para o céu'},
 {'score': 0.07594423741102219,
  'token': 17664,
  'token_str': 'espelho',
  'sequence': 'O rapaz olhou para o espelho'},
 {'score': 0.05604296550154686,
  'token': 119,
  'token_str': '.',
  'sequence': 'O rapaz olhou para o.'},
 {'score': 0.05529683828353882,
  'token': 1341,
  'token_str': 'lado',
  'sequence': 'O rapaz olhou para o lado'}]

In [23]:
fill_mask("A moça olhou para o [MASK] ")

[{'score': 0.1990056186914444,
  'token': 8242,
  'token_str': 'céu',
  'sequence': 'A moça olhou para o céu'},
 {'score': 0.12300949543714523,
  'token': 2979,
  'token_str': 'alto',
  'sequence': 'A moça olhou para o alto'},
 {'score': 0.09690188616514206,
  'token': 17664,
  'token_str': 'espelho',
  'sequence': 'A moça olhou para o espelho'},
 {'score': 0.07133549451828003,
  'token': 1341,
  'token_str': 'lado',
  'sequence': 'A moça olhou para o lado'},
 {'score': 0.05967789515852928,
  'token': 528,
  'token_str': 'mar',
  'sequence': 'A moça olhou para o mar'}]

In [24]:
fill_mask("Comprou uma [MASK] na loja. ")

[{'score': 0.1183428093791008,
  'token': 18099,
  'token_str': 'cerveja',
  'sequence': 'Comprou uma cerveja na loja.'},
 {'score': 0.05258983373641968,
  'token': 6997,
  'token_str': 'máquina',
  'sequence': 'Comprou uma máquina na loja.'},
 {'score': 0.043195273727178574,
  'token': 576,
  'token_str': 'vez',
  'sequence': 'Comprou uma vez na loja.'},
 {'score': 0.029871679842472076,
  'token': 7924,
  'token_str': 'camisa',
  'sequence': 'Comprou uma camisa na loja.'},
 {'score': 0.025105634704232216,
  'token': 4320,
  'token_str': 'peça',
  'sequence': 'Comprou uma peça na loja.'}]

In [25]:
fill_mask("A mulher não é [MASK]. ")

[{'score': 0.05108920857310295,
  'token': 12222,
  'token_str': 'casada',
  'sequence': 'A mulher não é casada.'},
 {'score': 0.02863135002553463,
  'token': 13420,
  'token_str': 'perfeita',
  'sequence': 'A mulher não é perfeita.'},
 {'score': 0.026146886870265007,
  'token': 1016,
  'token_str': 'assim',
  'sequence': 'A mulher não é assim.'},
 {'score': 0.02392844669520855,
  'token': 1848,
  'token_str': 'importante',
  'sequence': 'A mulher não é importante.'},
 {'score': 0.021442610770463943,
  'token': 2606,
  'token_str': 'mulher',
  'sequence': 'A mulher não é mulher.'}]

In [26]:
fill_mask("O homem não é [MASK]. ")

[{'score': 0.13655108213424683,
  'token': 13380,
  'token_str': 'perfeito',
  'sequence': 'O homem não é perfeito.'},
 {'score': 0.035791195929050446,
  'token': 4209,
  'token_str': 'igual',
  'sequence': 'O homem não é igual.'},
 {'score': 0.027270695194602013,
  'token': 3874,
  'token_str': 'nada',
  'sequence': 'O homem não é nada.'},
 {'score': 0.023961257189512253,
  'token': 1016,
  'token_str': 'assim',
  'sequence': 'O homem não é assim.'},
 {'score': 0.02120654098689556,
  'token': 4481,
  'token_str': 'humano',
  'sequence': 'O homem não é humano.'}]

In [27]:
fill_mask("Ele é um bom [MASK]. ")

[{'score': 0.12060219049453735,
  'token': 2357,
  'token_str': 'jogador',
  'sequence': 'Ele é um bom jogador.'},
 {'score': 0.11693714559078217,
  'token': 1416,
  'token_str': 'exemplo',
  'sequence': 'Ele é um bom exemplo.'},
 {'score': 0.04785142466425896,
  'token': 4022,
  'token_str': 'técnico',
  'sequence': 'Ele é um bom técnico.'},
 {'score': 0.04118023067712784,
  'token': 3620,
  'token_str': 'ator',
  'sequence': 'Ele é um bom ator.'},
 {'score': 0.025413991883397102,
  'token': 6325,
  'token_str': 'empresário',
  'sequence': 'Ele é um bom empresário.'}]

In [28]:
fill_mask("Ela é uma boa [MASK]. ")

[{'score': 0.162946879863739,
  'token': 3557,
  'token_str': 'atriz',
  'sequence': 'Ela é uma boa atriz.'},
 {'score': 0.13586226105690002,
  'token': 2760,
  'token_str': 'pessoa',
  'sequence': 'Ela é uma boa pessoa.'},
 {'score': 0.056161317974328995,
  'token': 8932,
  'token_str': 'amiga',
  'sequence': 'Ela é uma boa amiga.'},
 {'score': 0.034564364701509476,
  'token': 8208,
  'token_str': 'opção',
  'sequence': 'Ela é uma boa opção.'},
 {'score': 0.026774127036333084,
  'token': 2196,
  'token_str': 'mãe',
  'sequence': 'Ela é uma boa mãe.'}]

In [29]:
fill_mask("Faz de conta que ainda é [MASK]. ")

[{'score': 0.18625508248806,
  'token': 8545,
  'token_str': 'cedo',
  'sequence': 'Faz de conta que ainda é cedo.'},
 {'score': 0.14176622033119202,
  'token': 1695,
  'token_str': 'pouco',
  'sequence': 'Faz de conta que ainda é pouco.'},
 {'score': 0.08024295419454575,
  'token': 4931,
  'token_str': 'candidato',
  'sequence': 'Faz de conta que ainda é candidato.'},
 {'score': 0.06291192024946213,
  'token': 1373,
  'token_str': 'tarde',
  'sequence': 'Faz de conta que ainda é tarde.'},
 {'score': 0.061192095279693604,
  'token': 2656,
  'token_str': 'jovem',
  'sequence': 'Faz de conta que ainda é jovem.'}]

In [30]:
fill_mask("Depois da tempestade vem a [MASK]. ")

[{'score': 0.20495471358299255,
  'token': 12299,
  'token_str': 'neve',
  'sequence': 'Depois da tempestade vem a neve.'},
 {'score': 0.2026919424533844,
  'token': 8742,
  'token_str': 'seca',
  'sequence': 'Depois da tempestade vem a seca.'},
 {'score': 0.05817798152565956,
  'token': 11062,
  'token_str': 'fome',
  'sequence': 'Depois da tempestade vem a fome.'},
 {'score': 0.0526474192738533,
  'token': 9856,
  'token_str': 'chuva',
  'sequence': 'Depois da tempestade vem a chuva.'},
 {'score': 0.04578608274459839,
  'token': 13943,
  'token_str': 'lua',
  'sequence': 'Depois da tempestade vem a lua.'}]

In [31]:
fill_mask("Mais vale um pássaro na mão do que [MASK] voando. ")

[{'score': 0.3619248867034912,
  'token': 222,
  'token_str': 'um',
  'sequence': 'Mais vale um pássaro na mão do que um voando.'},
 {'score': 0.14355412125587463,
  'token': 6384,
  'token_str': 'alguém',
  'sequence': 'Mais vale um pássaro na mão do que alguém voando.'},
 {'score': 0.059984248131513596,
  'token': 1342,
  'token_str': 'outro',
  'sequence': 'Mais vale um pássaro na mão do que outro voando.'},
 {'score': 0.025123020634055138,
  'token': 14934,
  'token_str': 'voar',
  'sequence': 'Mais vale um pássaro na mão do que voar voando.'},
 {'score': 0.020025081932544708,
  'token': 2765,
  'token_str': 'estar',
  'sequence': 'Mais vale um pássaro na mão do que estar voando.'}]

In [32]:
fill_mask("Não tinha [MASK], mas almoçou mesmo assim. ")

[{'score': 0.05779163911938667,
  'token': 596,
  'token_str': 'tempo',
  'sequence': 'Não tinha tempo, mas almoçou mesmo assim.'},
 {'score': 0.03968477621674538,
  'token': 3382,
  'token_str': 'visto',
  'sequence': 'Não tinha visto, mas almoçou mesmo assim.'},
 {'score': 0.03311473876237869,
  'token': 13746,
  'token_str': 'namorado',
  'sequence': 'Não tinha namorado, mas almoçou mesmo assim.'},
 {'score': 0.032572563737630844,
  'token': 11062,
  'token_str': 'fome',
  'sequence': 'Não tinha fome, mas almoçou mesmo assim.'},
 {'score': 0.03244439885020256,
  'token': 3495,
  'token_str': 'dinheiro',
  'sequence': 'Não tinha dinheiro, mas almoçou mesmo assim.'}]

In [33]:
fill_mask("Bebeu um copo d'água, pois tinha [MASK]. ")

[{'score': 0.21553871035575867,
  'token': 11062,
  'token_str': 'fome',
  'sequence': "Bebeu um copo d'água, pois tinha fome."},
 {'score': 0.17282640933990479,
  'token': 7672,
  'token_str': 'medo',
  'sequence': "Bebeu um copo d'água, pois tinha medo."},
 {'score': 0.09888448566198349,
  'token': 14594,
  'token_str': 'febre',
  'sequence': "Bebeu um copo d'água, pois tinha febre."},
 {'score': 0.0951802134513855,
  'token': 4145,
  'token_str': 'razão',
  'sequence': "Bebeu um copo d'água, pois tinha razão."},
 {'score': 0.07313451170921326,
  'token': 11334,
  'token_str': 'sono',
  'sequence': "Bebeu um copo d'água, pois tinha sono."}]

In [34]:
fill_mask("Sem saber, ele descobriu um [MASK]. ")

[{'score': 0.1456645429134369,
  'token': 11021,
  'token_str': 'segredo',
  'sequence': 'Sem saber, ele descobriu um segredo.'},
 {'score': 0.07130926102399826,
  'token': 16269,
  'token_str': 'mistério',
  'sequence': 'Sem saber, ele descobriu um mistério.'},
 {'score': 0.05286533758044243,
  'token': 7441,
  'token_str': 'erro',
  'sequence': 'Sem saber, ele descobriu um erro.'},
 {'score': 0.03616425022482872,
  'token': 3420,
  'token_str': 'caminho',
  'sequence': 'Sem saber, ele descobriu um caminho.'},
 {'score': 0.024344172328710556,
  'token': 3204,
  'token_str': 'plano',
  'sequence': 'Sem saber, ele descobriu um plano.'}]

In [35]:
fill_mask("Pedro fez fortuna vendendo [MASK]. ")

[{'score': 0.1373293548822403,
  'token': 9701,
  'token_str': 'cavalos',
  'sequence': 'Pedro fez fortuna vendendo cavalos.'},
 {'score': 0.07332227379083633,
  'token': 7309,
  'token_str': 'drogas',
  'sequence': 'Pedro fez fortuna vendendo drogas.'},
 {'score': 0.07214704900979996,
  'token': 2978,
  'token_str': 'livros',
  'sequence': 'Pedro fez fortuna vendendo livros.'},
 {'score': 0.04423225298523903,
  'token': 6346,
  'token_str': 'discos',
  'sequence': 'Pedro fez fortuna vendendo discos.'},
 {'score': 0.03852641209959984,
  'token': 6401,
  'token_str': 'carros',
  'sequence': 'Pedro fez fortuna vendendo carros.'}]

In [36]:
fill_mask("Sem medo de ser [MASK]. ")

[{'score': 0.0936732366681099,
  'token': 8540,
  'token_str': 'feliz',
  'sequence': 'Sem medo de ser feliz.'},
 {'score': 0.053517743945121765,
  'token': 5139,
  'token_str': 'preso',
  'sequence': 'Sem medo de ser preso.'},
 {'score': 0.04083995893597603,
  'token': 4807,
  'token_str': 'morto',
  'sequence': 'Sem medo de ser morto.'},
 {'score': 0.03177829459309578,
  'token': 13205,
  'token_str': 'demitido',
  'sequence': 'Sem medo de ser demitido.'},
 {'score': 0.02574366144835949,
  'token': 13571,
  'token_str': 'criticado',
  'sequence': 'Sem medo de ser criticado.'}]